## Scrape Public Website to Generate Data Frame of Location Information - Toronto Neighborhoods

Using the wiki page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M generate a pandas dataframe.
Index + Columns: PostalCode, Borough, Neighborhood

Notes on Data Wrangling:
1. Only postal codes with the assigned burrough will be included in the dataset.  Burrough = Not Assigned or Null will not be included.
2. Neighborhood to PostalCode is M:1.  Many neighborhoods can exist in a single zipcode.
3. PostalCodes with more than one neighborhood will be combined in a single record.  All neighborhoods within the postalcode will be combined into the Neighborhood column for that record as a comma seperated list.
4. If a record has Borough that has Not assigned as a Neighborhood, the Borough Name will be replicated to the Neighborhood column.

The result shall be a Pandas Dataframe Containing Data Columns and a # of Records as determined by the reporting from python on the dataframe.



### Install HTML Parser Packages

In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

### Make an HTML Request to Get Wikipedia Content, Store to BeautifulSoup Container

In [3]:
import requests
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup=BeautifulSoup(website_url, 'lxml')
#print(soup.prettify())         #Visual check of content grab

### Process BeautifulSoup into Data Table - Extract the Raw Data

In [4]:
#Grab the table with the information
location_info_raw=soup.find('table', {'class':'wikitable sortable'})
#location_info_raw

#Create list of all the rows to process in the table
rows = location_info_raw.tbody.find_all('tr')
#print(rows[0:3])

#Extract the Column Headers from the th row
headings= [] #Create empty list for column headers
for th in rows[0].find_all("th"):    #Extract text from each th and add to the headings variable.  Make sure to strip special char.
    headings.append(th.text.replace('\n', ' ').strip())    
print(headings)

#Extract the Data Elements from the td cells in each row, skipping the first tr that contains the headers
table_data_rows = []  # Create empty list of table data
for table_row in rows[1:]:
    #print(table_row)
    columns=table_row.find_all("td")
    row_content=[]
    for column in columns:
        row_content.append(column.text.replace('\n', " ").strip())
    table_data_rows.append(row_content)
print(table_data_rows[0:5])        

['Postal Code', 'Borough', 'Neighbourhood']
[['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']]


### Create the Pandas Dataframe from the raw data

In [67]:
import pandas as pd
import numpy as np
df=pd.DataFrame(table_data_rows, columns=headings)

print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Clean the Data Frame Values

In [92]:
#Remove Records with Borough values = Not assigned
df.drop(df[(df['Borough'] == 'Not assigned')].index, inplace = True) 
df.shape

(103, 3)

In [104]:
#Add Borough Name to Any Unassigned Neighborhood Names
#df.nunique(axis=1)

#Find Neighborhoods labelled unassigned.
unassigned_list=[]
for ind in df.index:
    if df['Neighbourhood'][ind] == "Not assigned":
        unassigned_list.append([df['Postal Code'][ind], df['Borough'][ind]])
print("There are {} unassigned neighbourhood names.".format(len(unassigned_list)))

#Find duplicate Postal Code instances.  Find their index and combine their neighborhood values.
zip_list=[]
appears_multiple=[]

for ind in df.index:
    if df['Postal Code'][ind] not in zip_list:
        zip_list.append(df['Postal Code'][ind])
    else: 
        appears_multiple.append(df['Postal Code'][ind])
    #print(df['Postal Code'][ind], df['Borough'][ind]) 
    
print("There are {} duplicate postal codes.".format(len(appears_multiple)))

There are 0 unassigned neighbourhood names.
There are 0 duplicate postal codes.


A note about multiple postal codes vs instructions.  In the dataset each zip code is unique.  There are not situations where multiple neighborhoods need to be added to the same zip code to eliminate redundancy.   Someone on the wiki has already combined the neighbourhoods into their Postal Code list

In [105]:
df.shape

(103, 3)